В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [15]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [16]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

Кроме датасета glove-wiki-gigaword-100 есть еще:

conceptnet-numberbatch-17-06-300, fasttext-wiki-news-subwords-300, glove-twitter-100, glove-twitter-200, glove-twitter-25, glove-twitter-50, glove-wiki-gigaword-50, glove-wiki-gigaword-200, glove-wiki-gigaword-300, word2vec-ruscorpora-300, word2vec-google-news-300.

Датасеты:
1) word2vec-google-news-300: источник - Google News Dataset, объем - около 100 млрд. слов, использование - анализ новостных текстов, например, поиск ключевых слов и тем.

2) glove-twitter-25: источник - Twitter, объем: 2 миллиарда твитов, 27 миллиардов токенов, использование - сентимент-анализ, также поиск ключевых сллов.

3) glove-wiki-gigaword-50: Источник - Wikipedia 2014 и Gigaword 5, объем - 6 миллиардов токенов, использование - обучение ботов для генерации текста.

**Базовые операции с векторами**

In [17]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [18]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [19]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

w2v_model = api.load("glove-twitter-50")
print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 1193514
Размерность векторов: 50


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [20]:
anyword = input()

neighbours = w2v_model.most_similar(anyword, topn=10)
print(f"Слова, похожие на {anyword}:")
for word, score in neighbours:
    print(f"  {word}: {score:.4f}")

child
Слова, похожие на child:
  children: 0.8905
  mother: 0.8811
  father: 0.8722
  death: 0.8624
  woman: 0.8568
  birth: 0.8520
  daughter: 0.8485
  wife: 0.8210
  husband: 0.8189
  kids: 0.8176


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [28]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [33]:
model =  cbow_model = Word2Vec(
    cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)

In [34]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [35]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [36]:
# Найдите слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Найдите слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [37]:
ft_model = FastText(
    cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [38]:
try:
    similar = ft_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622
Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944
Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [39]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('learning')
compare_models('neural')


Сравнение для слова: 'learning'
  Word2Vec: слово не найдено
  FastText: ['духовка', 'пирог']

Сравнение для слова: 'neural'
  Word2Vec: слово не найдено
  FastText: ['мука', 'травы']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [48]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")
    print()

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']

  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']

  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']



In [41]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [42]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [46]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
print()
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks

Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [44]:
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [47]:
compare_documents(1, 0) # самый похожий получился
print()
compare_documents(1, 2)
print()
compare_documents(1, 3)
print()
compare_documents(1, 4)

Схожесть doc_1 и doc_0: 0.2735
  doc_1: deep learning uses neural networks
  doc_0: machine learning is interesting

Схожесть doc_1 и doc_2: -0.0573
  doc_1: deep learning uses neural networks
  doc_2: python programming for data science

Схожесть doc_1 и doc_3: 0.2031
  doc_1: deep learning uses neural networks
  doc_3: artificial intelligence is amazing

Схожесть doc_1 и doc_4: -0.2546
  doc_1: deep learning uses neural networks
  doc_4: computer vision processes images


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [52]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# размерность 10
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2
)
print()
print("Размерность: 10")
try:
    similar = ft_model.wv.most_similar('мясо', topn=5)
    print("Слова, похожие на 'мясо':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")

except KeyError:
    print("Слово 'мясо' не найдено в словаре")
print()
try:
    similar = ft_model.wv.most_similar('парить', topn=5)
    print("Слова, похожие на 'парить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'парить' не найдено в словаре")
print()
try:
    similar = ft_model.wv.most_similar('чай', topn=5)
    print("Слова, похожие на 'чай':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'чай' не найдено в словаре")

# размерность 50
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)
print()
print("Размерность: 50")
try:
    similar = ft_model.wv.most_similar('мясо', topn=5)
    print("Слова, похожие на 'мясо':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'мясо' не найдено в словаре")
print()
try:
    similar = ft_model.wv.most_similar('парить', topn=5)
    print("Слова, похожие на 'парить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'парить' не найдено в словаре")
print()
try:
    similar = ft_model.wv.most_similar('чай', topn=5)
    print("Слова, похожие на 'чай':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'чай' не найдено в словаре")

# размерность 100
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2
)
print()
print("Размерность: 100")
try:
    similar = ft_model.wv.most_similar('мясо', topn=5)
    print("Слова, похожие на 'мясо':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'мясо' не найдено в словаре")
print()
try:
    similar = ft_model.wv.most_similar('парить', topn=5)
    print("Слова, похожие на 'парить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'парить' не найдено в словаре")
print()
try:
    similar = ft_model.wv.most_similar('чай', topn=5)
    print("Слова, похожие на 'чай':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'чай' не найдено в словаре")


Размерность: 10
Слова, похожие на 'мясо':
  чай: 0.6647
  паста: 0.6470
  сахар: 0.6362
  здоровое: 0.6061
  тост: 0.5186

Слова, похожие на 'парить':
  овощи: 0.7486
  варить: 0.7291
  жарить: 0.6671
  духовка: 0.5900
  мука: 0.5879

Слова, похожие на 'чай':
  мясо: 0.6647
  салат: 0.5885
  горшок: 0.5081
  здоровое: 0.4640
  вода: 0.3456

Размерность: 50
Слова, похожие на 'мясо':
  соль: 0.2890
  яичница: 0.2870
  барбекю: 0.2247
  травы: 0.1998
  сахар: 0.1880

Слова, похожие на 'парить':
  жарить: 0.5047
  варить: 0.4805
  месить: 0.2541
  мариновать: 0.2352
  ингредиенты: 0.2190

Слова, похожие на 'чай':
  дрожжи: 0.2643
  гриль: 0.2594
  салат: 0.2340
  начинка: 0.2298
  хлеб: 0.2239

Размерность: 100
Слова, похожие на 'мясо':
  чай: 0.6647
  паста: 0.6470
  сахар: 0.6362
  здоровое: 0.6061
  тост: 0.5186

Слова, похожие на 'парить':
  овощи: 0.7486
  варить: 0.7291
  жарить: 0.6671
  духовка: 0.5900
  мука: 0.5879

Слова, похожие на 'чай':
  мясо: 0.6647
  салат: 0.5885
  горшо